In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import pickle
import os
import random
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
from torchvision import transforms


seed = 0
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
random.seed(seed)


# Preparing Dataset

In [ ]:
with open('/content/drive/MyDrive/Automn 2021/Machin Learning/Kaggle/fall2021-inf8245e-machine-learning/y_train.pkl', 'rb') as f:
          label_train = pickle.load(f)

with open('/content/drive/MyDrive/Automn 2021/Machin Learning/Kaggle/fall2021-inf8245e-machine-learning/x_train.pkl', 'rb') as f:
          x_train = pickle.load(f)

In [ ]:
with open('/content/drive/MyDrive/Automn 2021/Machin Learning/Kaggle/fall2021-inf8245e-machine-learning/x_test.pkl', 'rb') as f:
          x_test = pickle.load(f)

In [ ]:
x_train=np.array(x_train)
x_train=x_train/255
x_train_fl=x_train.reshape(11887,9216)


In [ ]:
x_train[1]

array([[0.69019608, 0.68235294, 0.68235294, ..., 0.66666667, 0.65882353,
        0.65098039],
       [0.6745098 , 0.67058824, 0.67058824, ..., 0.65098039, 0.64313725,
        0.63921569],
       [0.67843137, 0.6745098 , 0.67058824, ..., 0.65098039, 0.64313725,
        0.62745098],
       ...,
       [0.69411765, 0.63921569, 0.51372549, ..., 0.70980392, 0.72941176,
        0.7372549 ],
       [0.67058824, 0.6       , 0.50588235, ..., 0.67843137, 0.68627451,
        0.69411765],
       [0.64313725, 0.57254902, 0.53333333, ..., 0.66666667, 0.66666667,
        0.69019608]])

In [ ]:
std=x_train.std()
mean=x_train.mean()
print(std,mean)

0.25659970028300466 0.4876348845146479


In [ ]:
x_test=np.array(x_test)
x_test=x_test/255

x_test_fl=x_test.reshape(17831,9216)
x_test_df = pd.DataFrame(x_test_fl)

In [ ]:
class_dic = { "big_cats":0,  "butterfly":1,  "cat":2,  "chicken":3,  "cow":4, "dog":5,  "elephant":6, "goat":7,  "horse":8,  "spider":9,  "squirrel":10}
df = pd.DataFrame()
df['Y']=label_train
label_train= df['Y'].map(class_dic).values
label_train

array([ 0,  5, 10, ...,  6,  5,  4])

In [ ]:
x_train_df = pd.DataFrame(x_train_fl)
x_train_df['label']=label_train
x_train_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9177,9178,9179,9180,9181,9182,9183,9184,9185,9186,9187,9188,9189,9190,9191,9192,9193,9194,9195,9196,9197,9198,9199,9200,9201,9202,9203,9204,9205,9206,9207,9208,9209,9210,9211,9212,9213,9214,9215,label
0,0.113725,0.121569,0.329412,0.372549,0.443137,0.419608,0.427451,0.443137,0.517647,0.427451,0.400000,0.400000,0.368627,0.341176,0.415686,0.505882,0.490196,0.541176,0.517647,0.466667,0.521569,0.560784,0.682353,0.580392,0.450980,0.396078,0.466667,0.411765,0.384314,0.317647,0.278431,0.239216,0.329412,0.380392,0.376471,0.384314,0.325490,0.294118,0.313725,0.513725,...,0.403922,0.396078,0.368627,0.360784,0.345098,0.333333,0.180392,0.188235,0.192157,0.188235,0.298039,0.282353,0.352941,0.325490,0.317647,0.278431,0.400000,0.521569,0.607843,0.513725,0.666667,0.721569,0.686275,0.694118,0.627451,0.678431,0.607843,0.435294,0.400000,0.396078,0.364706,0.392157,0.400000,0.607843,0.301961,0.129412,0.203922,0.501961,0.360784,0
1,0.690196,0.682353,0.682353,0.698039,0.701961,0.694118,0.701961,0.701961,0.709804,0.709804,0.717647,0.729412,0.737255,0.721569,0.725490,0.725490,0.733333,0.741176,0.733333,0.729412,0.717647,0.713725,0.713725,0.721569,0.737255,0.749020,0.764706,0.784314,0.788235,0.803922,0.815686,0.819608,0.831373,0.850980,0.847059,0.839216,0.843137,0.827451,0.827451,0.831373,...,0.082353,0.152941,0.345098,0.364706,0.470588,0.537255,0.360784,0.290196,0.317647,0.364706,0.474510,0.619608,0.690196,0.764706,0.780392,0.772549,0.749020,0.749020,0.749020,0.752941,0.760784,0.756863,0.737255,0.729412,0.741176,0.713725,0.709804,0.705882,0.701961,0.682353,0.666667,0.666667,0.643137,0.623529,0.654902,0.662745,0.666667,0.666667,0.690196,5
2,0.392157,0.286275,0.219608,0.219608,0.156863,0.180392,0.227451,0.278431,0.270588,0.149020,0.149020,0.133333,0.180392,0.203922,0.188235,0.149020,0.156863,0.200000,0.243137,0.294118,0.360784,0.392157,0.407843,0.427451,0.431373,0.372549,0.360784,0.301961,0.282353,0.298039,0.329412,0.352941,0.368627,0.407843,0.403922,0.372549,0.325490,0.270588,0.129412,0.023529,...,0.701961,0.733333,0.411765,0.435294,0.572549,0.678431,0.498039,0.396078,0.705882,0.129412,0.309804,0.419608,0.364706,0.337255,0.313725,0.470588,0.282353,0.411765,0.611765,0.662745,0.223529,0.254902,0.411765,0.674510,0.654902,0.705882,0.709804,0.443137,0.556863,0.525490,0.423529,0.258824,0.513725,0.517647,0.545098,0.384314,0.513725,0.352941,0.341176,10
3,0.231373,0.231373,0.231373,0.231373,0.231373,0.231373,0.231373,0.231373,0.243137,0.243137,0.243137,0.239216,0.239216,0.239216,0.239216,0.235294,0.227451,0.219608,0.215686,0.215686,0.211765,0.200000,0.196078,0.192157,0.188235,0.180392,0.176471,0.160784,0.152941,0.145098,0.141176,0.129412,0.125490,0.117647,0.113725,0.109804,0.101961,0.094118,0.094118,0.090196,...,0.247059,0.243137,0.239216,0.235294,0.235294,0.231373,0.227451,0.227451,0.231373,0.227451,0.223529,0.219608,0.223529,0.223529,0.223529,0.227451,0.223529,0.223529,0.223529,0.223529,0.227451,0.227451,0.231373,0.235294,0.231373,0.235294,0.239216,0.239216,0.243137,0.247059,0.254902,0.250980,0.254902,0.262745,0.266667,0.278431,0.274510,0.278431,0.294118,9
4,0.137255,0.133333,0.133333,0.145098,0.117647,0.137255,0.156863,0.149020,0.168627,0.145098,0.152941,0.160784,0.156863,0.160784,0.160784,0.156863,0.168627,0.156863,0.176471,0.180392,0.184314,0.176471,0.180392,0.184314,0.184314,0.184314,0.184314,0.188235,0.192157,0.196078,0.196078,0.200000,0.192157,0.192157,0.184314,0.184314,0.184314,0.188235,0.196078,0.200000,...,0.819608,0.784314,0.729412,0.654902,0.639216,0.803922,0.721569,0.729412,0.843137,0.870588,0.741176,0.858824,0.862745,0.811765,0.760784,0.792157,0.764706,0.713725,0.768627,0.721569,0.745098,0.694118,0.698039,0.650980,0.690196,0.694118,0.627451,0.603922,0.541176,0.600000,0.525490,0.513725,0.564706,0.470588,0.470588,0.501961,0.396078,0.423529,0.447059,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [ ]:
#splitting train,test,valid
train, validate, test = np.split(x_train_df.sample(frac=1,random_state=np.random.RandomState(seed)), [int(.8*len(x_train_df)), int(.9*len(x_train_df))])

In [ ]:
#combining train and vcalidation dataset
train_valid=pd.concat([train,validate], ignore_index=True)

# Gradient Boosting

In [ ]:
#Gradient Boosting on combination of validation and train dataset
clf_grad_boost = GradientBoostingClassifier(random_state=0).fit(train_valid.iloc[:,:9216], train_valid.iloc[:,9216:].to_numpy().T[0])


In [ ]:
f1_train = f1_score( list(train['label']),clf_grad_boost.predict(train.iloc[:,:9216]), average='macro')
f1_valid = f1_score( list(validate['label']),clf_grad_boost.predict(validate.iloc[:,:9216]), average='macro')
f1_test = f1_score( list(test['label']),clf_grad_boost.predict(test.iloc[:,:9216]), average='macro')
print(f'F1-score of Gradient boosting \n Training: {f1_train}\n Validation: {f1_valid} \n Test:{f1_test}')

F1-score of Gradient boosting 
 Training: 0.7803119697726686
 Validation: 0.7817153325066672 
 Test:0.25658316232439643


In [ ]:
y_test_hat=clf_grad_boost.predict(x_test_df)
sub = pd.DataFrame()
sub['id'] = np.arange(17831)
sub['class'] =y_test_hat
sub.to_csv("/content/drive/MyDrive/Automn 2021/Machin Learning/Kaggle/fall2021-inf8245e-machine-learning/sub_gradient_boost.csv",index=False)
sub.head(20)

,id,class
0,0,3
1,1,9
2,2,4
3,3,3
4,4,5
5,5,5
6,6,9
7,7,5
8,8,9
9,9,8


# SVM and hyperparametr tunning

In [ ]:
#SVM on train dataset
clf_svc=SVC().fit(train.iloc[:,:9216], train.iloc[:,9216:].to_numpy().T[0])

In [ ]:

f1_train = f1_score( list(train['label']),clf_svc.predict(train.iloc[:,:9216]), average='micro')
f1_valid = f1_score( list(validate['label']),clf_svc.predict(validate.iloc[:,:9216]), average='micro')
f1_test = f1_score( list(test['label']),clf_svc.predict(test.iloc[:,:9216]), average='micro')
print(f'F1-score of SVM \n Training: {f1_train}\n Validation: {f1_valid} \n Test:{f1_test}')

F1-score of Gradient boosting 
 Training: 0.5712124152402475
 Validation: 0.25294668532438497 
 Test:0.2565482493552381


In [ ]:
# params = {'penalty':['l1','l2'], 'C':np.arange(0.1, 0.6, 0.1), 'loss':['hinge', 'squared_hinge'], 'dual':[True,False]}
# params = {'penalty':['l1', 'l2'], 'loss':['hinge', 'squared_hinge']}
params = {'C':np.arange(1, 100, 20)}
f = [-1 for i in range(train.shape[0])] + [1 for i in range(validate.shape[0])]
p= PredefinedSplit(test_fold = f)
clf_svc_h = GridSearchCV(SVC(), param_grid=params, cv=p)
clf_svc_h.fit(train_valid.iloc[:,:9216], train_valid['label'])

GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  1,  1])),
             estimator=SVC(), param_grid={'C': array([ 1, 21, 41, 61, 81])})

In [ ]:
clf_svc_h.best_params_

{'C': 21}

In [ ]:
f1_train = f1_score( list(train['label']),clf_svc_h.predict(train.iloc[:,:9216]), average='micro')
f1_valid = f1_score( list(validate['label']),clf_svc_h.predict(validate.iloc[:,:9216]), average='micro')
f1_test = f1_score( list(test['label']),clf_svc_h.predict(test.iloc[:,:9216]), average='micro')
print(f'F1-score of SVM with hyperparameter tunning \n Training: {f1_train}\n Validation: {f1_valid} \n Test:{f1_test}')

F1-score of SVM with hyperparameter tunning 
 Training: 0.999894836470712
 Validation: 1.0 
 Test:0.3700588730025231


In [ ]:
y_test_hat=clf_svc_h.predict(x_test_df)
sub = pd.DataFrame()
sub['id'] = np.arange(17831)
sub['class'] =y_test_hat
sub.to_csv("/content/drive/MyDrive/Automn 2021/Machin Learning/Kaggle/fall2021-inf8245e-machine-learning/sub.csv",index=False)
sub.head(20)

,id,class
0,0,10
1,1,0
2,2,8
3,3,5
4,4,3
5,5,4
6,6,5
7,7,5
8,8,9
9,9,6
